REGRESSION PROJECT NOTEBOOK

* This notebook can be updated by all in the team

In [5]:
#Lets have a look at our data 
import pandas as pd

df_test = pd.read_csv('df-test_set.csv', index_col=0)

display(df_test) #general overview
print(df_test.info()) #better insight into the datatype for each column

,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
Index,,,,,,,,,,,,
1,W.CAPE-BERGRIVER ETC,EC120,1M,12.0,APPLE GOLDEN DELICIOUS,2020-07-09,128.0,136.0,5008.0,38,456.0,0
2,W.CAPE-BERGRIVER ETC,M4183,1X,18.3,APPLE GOLDEN DELICIOUS,2020-01-20,220.0,220.0,1760.0,8,146.4,2
3,W.CAPE-BERGRIVER ETC,EC120,1S,12.0,APPLE GOLDEN DELICIOUS,2020-08-19,120.0,120.0,720.0,6,72.0,45
4,W.CAPE-BERGRIVER ETC,M4183,1M,18.3,APPLE GOLDEN DELICIOUS,2020-05-06,160.0,160.0,160.0,1,18.3,8
5,W.CAPE-BERGRIVER ETC,M4183,1L,18.3,APPLE GOLDEN DELICIOUS,2020-05-04,140.0,160.0,14140.0,100,1830.0,19
...,...,...,...,...,...,...,...,...,...,...,...,...
681,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,APPLE GOLDEN DELICIOUS,2020-05-26,30.0,30.0,1320.0,44,396.0,856
682,CAPE,EC120,1M,12.0,APPLE GOLDEN DELICIOUS,2020-01-10,50.0,132.0,6290.0,52,624.0,785
683,W.CAPE-BERGRIVER ETC,JE090,2X,9.0,APPLE GOLDEN DELICIOUS,2020-06-03,55.0,55.0,220.0,4,36.0,170


<class 'pandas.core.frame.DataFrame'>
Int64Index: 685 entries, 1 to 685
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province        685 non-null    object 
 1   Container       685 non-null    object 
 2   Size_Grade      685 non-null    object 
 3   Weight_Kg       685 non-null    float64
 4   Commodities     685 non-null    object 
 5   Date            685 non-null    object 
 6   Low_Price       685 non-null    float64
 7   High_Price      685 non-null    float64
 8   Sales_Total     685 non-null    float64
 9   Total_Qty_Sold  685 non-null    int64  
 10  Total_Kg_Sold   685 non-null    float64
 11  Stock_On_Hand   685 non-null    int64  
dtypes: float64(5), int64(2), object(5)
memory usage: 69.6+ KB
None


In [9]:
df_train = pd.read_csv('df-train_set.csv')

display(df_train)
print(df_train.info())

,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
0,CAPE,EC120,1L,12.0,APPLE GRANNY SMITH,2020-03-10,108.0,112.0,3236.0,29,348.0,0,9.30
1,CAPE,M4183,1L,18.3,APPLE GOLDEN DELICIOUS,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51
2,GAUTENG,AT200,1L,20.0,AVOCADO PINKERTON,2020-05-05,70.0,80.0,4860.0,66,1320.0,50,3.68
3,TRANSVAAL,BJ090,1L,9.0,TOMATOES-LONG LIFE,2020-01-20,60.0,60.0,600.0,10,90.0,0,6.67
4,WESTERN FREESTATE,PP100,1R,10.0,POTATO SIFRA (WASHED),2020-07-14,40.0,45.0,41530.0,927,9270.0,393,4.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64371,TRANSVAAL,AG100,1M,10.0,ONIONS MILD,2020-05-18,70.0,75.0,19695.0,281,2810.0,3583,7.01
64372,IMPORTS - OUTSIDE RSA,BC180,X,18.0,BANANA,2020-02-05,160.0,220.0,13560.0,72,1296.0,887,10.46
64373,TRANSVAAL,TS035,1M,3.5,TOMATOES-LONG LIFE,2020-08-20,20.0,20.0,6000.0,300,1050.0,900,5.71
64374,NATAL,BJ100,2M,10.0,LEMON,2020-08-18,50.0,80.0,16730.0,277,2770.0,372,6.04


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64376 entries, 0 to 64375
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Province          64376 non-null  object 
 1   Container         64376 non-null  object 
 2   Size_Grade        64376 non-null  object 
 3   Weight_Kg         64376 non-null  float64
 4   Commodities       64376 non-null  object 
 5   Date              64376 non-null  object 
 6   Low_Price         64376 non-null  float64
 7   High_Price        64376 non-null  float64
 8   Sales_Total       64376 non-null  float64
 9   Total_Qty_Sold    64376 non-null  int64  
 10  Total_Kg_Sold     64376 non-null  float64
 11  Stock_On_Hand     64376 non-null  int64  
 12  avg_price_per_kg  64376 non-null  float64
dtypes: float64(6), int64(2), object(5)
memory usage: 6.4+ MB
None


In [11]:
sample = pd.read_csv('sample_submission.csv')

sample #our end result should look somethng similar

,Index,avg_price_per_kg
0,1,13.94
1,2,1.30
